In [2]:
import sympy as sp
import string

In [10]:
ascii_greek = ['Α', 'α',
               'Β', 'β',
               'Γ', 'γ',
               'Δ', 'δ',
               'Ε', 'ε',
               'Ζ', 'ζ',
               'Η', 'η',
               'Θ', 'θ',
               'Ι', 'ι',
               'Κ', 'κ',
               'Λ', 'λ',
               'Μ', 'μ',
               'Ν', 'ν',
               'Ξ', 'ξ',
               'Ο', 'ο',
               'Π', 'π',
               'Ρ', 'ρ',
               'Σ', 'σ',
               'Τ', 'τ',
               'Υ', 'υ',
               'Φ', 'φ',
               'Χ', 'χ',
               'Ψ', 'ψ',
               'Ω', 'ω']
latex_greek = ['\\Alpha', '\\alpha',
               '\\Beta', '\\beta',
               '\\Gamma', '\\gamma',
               '\\Delta', '\\delta',
               '\\Epsilon', '\\epsilon',
               '\\Zeta', '\\zeta',
               '\\Eta', '\\eta',
               '\\Theta', '\\theta',
               '\\Iota', '\\iota',
               '\\Kappa', '\\kappa',
               '\\Lambda', '\\lambda',
               '\\Mu', '\\mu',
               '\\Nu', '\\nu',
               '\\Xi', '\\xi',
               '\\Omicron', '\\omicron',
               '\\Pi', '\\pi',
               '\\Rho', '\\rho',
               '\\Sigma', '\\sigma',
               '\\Tau', '\\tau',
               '\\Upsilon', '\\upsilon',
               '\\Phi', '\\phi',
               '\\Chi', '\\chi',
               '\\Psi', '\\psi',
               '\\Omega', '\\omega']
symbols = string.ascii_letters + ''.join(ascii_greek)

In [12]:
def transform_greek(s: str):
    greek_transform = dict(zip(latex_greek, ascii_greek))
    for letter in latex_greek:
        while letter in s:
            s = s.replace(letter, greek_transform[letter])
    return s

def transform_multiplication(s: str):
    length = len(s)
    i = 1
    while(i < length):
        if (s[i] in symbols and s[i - 1] in (symbols + '1234567890')):
            s = s[:i] + '*' + s[i:]
            length += 1
        i += 1